# Automated Matrix Inversion
This notebook has an automated version of the matrix inversion through row manipulation presented previously.

https://docs.sympy.org/latest/tutorial/calculus.html

https://www.tutorialspoint.com/sympy/sympy_plotting.htm

https://docs.sympy.org/latest/modules/plotting.html#plotgrid-class

In [1]:
from sympy import *
import matplotlib.pyplot as plt
init_session()
init_printing(use_unicode=True)

from copy import deepcopy

from IPython.display import display, Math, Latex

# remove/comment line below to get plots in a seperate window
%matplotlib inline

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



Below, we define three routines that manipulate matrices

ShowM(M1, M2, ...) just prints the matrices M1, M2, ... (you can add as many as you like

RowManip(factor, startRow, targetRow, M1, M2, ...)
adds factor * startRow to target row, for all matrices M1, M2, ....
(make sure that these rows both exist, otherwise you'll get a runtime error)

RowSwap(row1, row1, M1, M2, ...)
swaps rows 1 and 2 in matrices M1, M2, ...


In [2]:
def ShowMSimple(*matrices):
    for M in matrices:
        for row in M:
            print(row)
        print('')

def ShowM(*matrices):
    string="\(";    
    for M in matrices:
        Mstr = "   \\begin{pmatrix} "
        for row in M:
            doAmp=False;
            for a in row:
                if doAmp: Mstr += " & "
                else: doAmp = True
                Mstr += str(a)
            Mstr += " \\\\ "
        Mstr += " \\end{pmatrix} \;\;\;\;\;"
        string += Mstr
    string += "\)"
    # print(string)
    display(Latex(string))

    
# Note: Now that we'll use our routines internally,
# we number our matrix rows and columns the pyton way
# i.e. 0, 1, 2, ...
def RowManip(factor, sr, tr, *matrices):
    for M in matrices:
        for i in range(0, len(M[tr])):
            M[tr][i] = M[tr][i]+factor*M[sr][i]
            
# Note: Now that we'll use our routines internally,
# we number our matrix rows and columns  0, 1, 2, ...
def RowSwap(r1, r2, *matrices):
    for M in matrices:
        temprow = M[r2]
        M[r2]=M[r1]
        M[r1]=temprow
        
def MultiplyRow(factor, r, *matrices):
    for M in matrices:
        for i in range(0, len(M[r])):
            M[r][i] = factor*M[r][i]
            
def MultiplyMatrices(A, B):
    # I'm doing no checking of dimensions - make sure you pass sensible matrices
    # (or add a check yourself)
    #ShowM(A,B)
    rows = len(A)
    cols = len(B[0])
    C = [[0 for i in range(0,cols)] for j in range(0,rows)] # init results matrix
    
    for m in range(0, rows):
        for n in range(0, cols):
            for k in range(0, len(A[m])):
                C[m][n] += A[m][k] * B[k][n]
                   
    return C
            

def roundM(M_in, decimals=10):
    rows = len(M_in)
    cols = len(M_in[0])
    M = [[0 for i in range(0,rows)] for j in range(0,cols)] # init results matrix

    for m in range(0, rows):
        for n in range(0, cols):
            M[m][n] = round(M_in[m][n], decimals)
    return M

Above we have the code for the previous examples, except that we changed the row and column numbering scheme to be more suitable for use in internal python routines (i.e. we start with 0 rather than 1). Below, we write a routine to automate the matrix inversion process that we previously did by hand.

In [3]:
def InvertMatrix(M_in, verbose=True):
    M = deepcopy(M_in) ## we won't manipulate M_in itself.
    
    ## we assume it's a nxn (i.e. square) matrix.
    dim = len(M)
    I  = [[0 for i in range(0, dim)] for i in range(0, dim)]
    for k in range(0,dim):
        I[k][k]=1
    
    if verbose:
        print("Our starting point is this")
        ShowM(M, I)
        
    for n in range(0, dim):  ## we are looping over columns
        ## first we find the largest element in current column 
        if verbose: print("Finding pivot in column ", n+1)
        maxi   = -1
        pivrow =  0
        for m in range(n, dim):
            ##print(abs(M[m][n]))
            if abs(M[m][n]) > maxi :
                maxi = abs(M[m][n])
                pivrow = m
                ##print(pivrow, ", ", maxi)
                
        if maxi == 0: 
            print("Singular matrix, cannot be inverted")
            return I
        
        ## found pivot - we will put this on the top
        ## of the sub-matrix we are concerned with:
        RowSwap(n, pivrow, M, I)
        if verbose: 
            print("Swapping rows ", n+1, ", ", pivrow +1," gives:")
            ShowM(M, I)
        
        ## now we row-manipulate all rows (except current pivot) to
        ## set all other elements in the current column to zero.
        if M[n][n] == 0: break;
        for m in range(0, dim):
            if n != m: RowManip(-M[m][n]/M[n][n], n, m, M, I)
        
        if verbose:
            print("Eliminating all elements in column ", n+1, " (except in row ", n+1, ", of course) gives:")
            ShowM(M, I)
        
    ## We now have a diagonal matrix.
    ## Now need to set those diagonal elements to 1
    if verbose: print("Now multilying rows to set diagonal elements to one:")
    for m in range(0, dim):
        d = M[m][m]
        if d != 0:
            MultiplyRow(1/d, m, M, I)
    
    if(verbose):
        ShowM(M, I)
        print("So, our inverse is:")
        ShowM(I)
        print("or, sometimes prittier, rounded to 10 decimal places:")
        ShowM(roundM(I))
        print("Trying it out:")
        C = MultiplyMatrices(M_in, I);
        ShowM(M_in, I)
        print("=")
        ShowM(roundM(C))
    
    return I

Let's now use it.

In [4]:
M33= [[1, 2, 3]
     ,[2, 3, 4]
     ,[2, 1, 1]]

InvertMatrix(M33)

Our starting point is this


<IPython.core.display.Latex object>

Finding pivot in column  1
Swapping rows  1 ,  2  gives:


<IPython.core.display.Latex object>

Eliminating all elements in column  1  (except in row  1 , of course) gives:


...

<IPython.core.display.Latex object>

Trying it out:


<IPython.core.display.Latex object>

=


<IPython.core.display.Latex object>

# Using matrix inverstion to solve equations:

How about using it to solve this system of equation:

$
\begin{align*}
 5 x_1 + 2 x_2 + 9 x_3 + x_4 &= 11 \\
 2 x_1 + 3 x_2 +10 x_3 + 2 x_4 &= 12 \\
   x_1 +   x_2 +   x_3 + 4 x_4 &= 13 \\
         6 x_2 +17 x_3 +   x_4 &= 14
\end{align*}
$

We start by writing this as

$
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{pmatrix}
=
\begin{pmatrix}
11 \\ 12 \\ 13 \\ 14
\end{pmatrix}
$

Now, if we find
$
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}^{-1}
$ 
we have solved the problem, because


$
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}^{-1}
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}
\begin{pmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{pmatrix}
=
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}^{-1}
\begin{pmatrix}
11 \\ 12 \\ 13 \\ 14
\end{pmatrix}
$

and hence

$
\begin{pmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{pmatrix}
=
\begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}^{-1}
\begin{pmatrix}
11 \\ 12 \\ 13 \\ 14
\end{pmatrix}
$



Let's create python versions of the matrix 
$ M_{44} = \begin{pmatrix}
 5 & 2 &  9 & 1 \\
 2 & 3 & 10 & 2 \\
 1 & 1 &  1 & 4  \\
 0 & 6 & 17 & 1
\end{pmatrix}
$ 
and the vector on the right hand side of the equation,
$
\begin{pmatrix}
11 \\ 12 \\ 13 \\ 14
\end{pmatrix}
$
.

In [5]:
M44= [[5, 2,  9, 1]
     ,[2, 3, 10, 2]
     ,[1, 1,  1, 4]
     ,[0, 6, 17, 1]]

b  = [[11]
    , [12]
    , [13]
    , [14]] ## the seemingly superfluous brackets are needed because all our routies expect matrices, 
            ## i.e. arrays of row vectors. [11] is a row with one element (i.e. OK),
            ## while 11 is not a row (our programmes will choke on this)

Now we invert $M_{44}$ using our programme:

In [6]:
M44_inverse = InvertMatrix(M44)

Our starting point is this


<IPython.core.display.Latex object>

Finding pivot in column  1
Swapping rows  1 ,  1  gives:


<IPython.core.display.Latex object>

Eliminating all elements in column  1  (except in row  1 , of course) gives:


...

<IPython.core.display.Latex object>

Trying it out:


<IPython.core.display.Latex object>

=


<IPython.core.display.Latex object>

Now we have that inverse, we can solve the equation

In [7]:
Sol = MultiplyMatrices(M44_inverse, b)
ShowM(Sol)

<IPython.core.display.Latex object>

which represents the solution vector $ \begin{pmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \end{pmatrix} $

## Apply the algorithm to the matrix you inverted yourself in the previous notebook

In [8]:
N33= [[6, 2, 1]
     ,[1, -6, -1]
     ,[-2, 1, 0]]

InvertMatrix(N33)

Our starting point is this


<IPython.core.display.Latex object>

Finding pivot in column  1
Swapping rows  1 ,  1  gives:


<IPython.core.display.Latex object>

Eliminating all elements in column  1  (except in row  1 , of course) gives:


...

<IPython.core.display.Latex object>

Trying it out:


<IPython.core.display.Latex object>

=


<IPython.core.display.Latex object>

## Playing around with the matrix inversion program:

What will it do with a singular matrix? Let's see:

In [9]:
O33= [[6, 6, 1]
     ,[1, 1, -1]
     ,[-2, -2, 0]]

InvertMatrix(O33)

Our starting point is this


<IPython.core.display.Latex object>

Finding pivot in column  1
Swapping rows  1 ,  1  gives:


<IPython.core.display.Latex object>

Eliminating all elements in column  1  (except in row  1 , of course) gives:


<IPython.core.display.Latex object>

Finding pivot in column  2
Singular matrix, cannot be inverted


## Improvements
Why not write your own version or improve the algorithm by adding functionality? One potential avenue would be to improve its accuracy by using sympy, which does arithmetic manipulations and can find solutions such as $\sqrt{2}$ and $1/3$, rather than approximating these as $1.41...$ and $0.333...$.